In [1]:
import pyspark
from pyspark.sql import SparkSession
from minio import Minio
from minio.error import S3Error
import pandas as pd
import numpy as np
import json
import io
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [2]:
spark = SparkSession.builder \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.driver.maxResultSize", "6g") \
    .config("spark.driver.memory", "3g") \
    .config("spark.executor.memory", "3g") \
    .appName("yelp_dataset") \
    .getOrCreate()

minio_client = Minio(
    "localhost:9000",
    access_key="sJ8IkEjav4gkDwjt2BxK",
    secret_key="atZpF6WhWSD0tH2vSxylDHYpdOWZ11zdnSZ87ca8",
    secure=False
)

try:
    buckets = minio_client.list_buckets()
    print("Buckets:")
    for bucket in buckets:
        print(bucket.name, bucket.creation_date)
except S3Error as e:
    print(f"Error occurred: {e}")

Buckets:
bronze 2024-11-09 00:38:23.211000+00:00


In [3]:
mongo_url = 'mongodb://127.0.0.1/'
bucket_yelp = "yelp-dataset"
bucket_bronze = "bronze"

In [4]:
collections = ['business', 'checkin', 'tip', 'user', 'review']
for collection in collections:
    try:
        print(f"Processing collection: {collection}")
        bronze_df = spark.read.format("com.mongodb.spark.sql.DefaultSource") \
            .option('spark.mongodb.input.uri', mongo_url) \
            .option('spark.mongodb.input.database', 'YELP') \
            .option('spark.mongodb.input.collection', collection) \
            .load()
            
        bronze_df.repartition(1).write.format("parquet").mode("append").save(f"./Parquet/{collection}")
        print(f"Successfully processed collection: {collection}")
    except Exception as e:
        print(f"Error processing collection {collection}: {e}")

Processing collection: business
Successfully processed collection: business
Processing collection: checkin
Successfully processed collection: checkin
Processing collection: tip
Successfully processed collection: tip
Processing collection: user
Successfully processed collection: user
Processing collection: review
Successfully processed collection: review


In [ ]:
import os
import shutil
for collection in collections:
    try:
        for file_name in os.listdir(f'./Parquet/{collection}'):
            if file_name.endswith(".parquet"):
                print(f"Processing collection: {collection}")
                os.rename(f'./Parquet/{collection}/{file_name}', f'./Parquet/{collection}/{collection}.parquet')
                print(f"Successfully renamed collection: {collection}")
    except Exception as e:
            print(f"Error renaming collection {collection}: {e}")

In [6]:
bucket_name = 'bronze'

for collection in collections:
    try:
        file = f'./Parquet/{collection}/{collection}.parquet'
        obj = f'{collection}.parquet'
        minio_client.fput_object(bucket_name, obj, file)
        print(f"Successfully pushed collection: {collection} to minio")
    except Exception as e:
        print(f"Error pushing collection {collection} to minio: {e}")

Successfully pushed collection: business to minio
Successfully pushed collection: checkin to minio
Successfully pushed collection: tip to minio
Successfully pushed collection: user to minio
Successfully pushed collection: review to minio


In [7]:
spark.stop()